In [1]:
import pandas as pd
import os
from sksurgerynditracker.nditracker import NDITracker
import numpy as np

In [ ]:
"""
    Initialize the NDI Polaris Optical tracker (Track the Calibration Plate)
    MAKE SURE you have the Chicken Foot .rom file and its path
"""
SETTINGS_CP = {
    "tracker type": "vega",
    "ip address": "169.254.7.250",
    "port": 8765,

    # Windows Path
    "romfiles": ["CALIBRATION PLATE .ROM FILE PATH"]
}
tracker_CP = NDITracker(SETTINGS_CP)
tracker_CP.start_tracking()

In [ ]:
"""
    Initialize the NDI Aurora EM tracker (Attached to Patient Body)
"""
SETTINGS_BODY = {
    "tracker type": "aurora",
    "port": "COM3"
}
tracker_BODY = NDITracker(SETTINGS_BODY)
tracker_BODY.start_tracking()

In [ ]:
# {PO} to {CP}, tracked by Polaris
_, _, _, PO_to_CP_calibration, _ = tracker_CP.get_frame()

In [ ]:
# {CP} to {PO}
CP_to_PO_calibration = np.linalg.inv(PO_to_CP_calibration)

In [ ]:
# {FG} to {CP}, calculated by Calibration_Plate_Aurora_Registration.ipynb
FG_to_CP_calibration = np.load("FG_to_CP_calibration.npy")

In [ ]:
# {FG} to {PO}
FG_to_PO = np.dot(FG_to_CP_calibration, CP_to_PO_calibration)

In [ ]:
# {PO} to {FG}
PO_to_FG = np.linalg.inv(FG_to_PO)
# Save PO_to_FG, fixed relationship
np.save("PO_to_FG.npy", PO_to_FG)

In [ ]:
# {FG} to {EM2} or Patient Body
_, _, _, FG_to_EM2_calibration, _ = tracker_BODY.get_frame() 

In [ ]:
# {EM2} or Patient Body to {FG}
EM2_to_FG_calibration = np.linalg.inv(FG_to_EM2_calibration)


In [ ]:
# {EM2} or Patient Body to {PO}
EM2_to_PO_calibration = np.dot(EM2_to_FG_calibration, FG_to_PO)

In [ ]:
# {PO} to {CT}, calculated by CT_Polaris_Registration.ipynb
PO_to_CT_calibration = np.load("PO_to_CT_calibration.npy")

In [ ]:
# {EM2} or Patient Body to {CT}, fixed relationship
EM2_to_CT = np.dot(EM2_to_PO_calibration, PO_to_CT_calibration)
# Save EM2_to_CT
np.save("EM2_to_CT.npy", EM2_to_CT)